In [11]:
# !apt-get update && apt-get install ffmpeg libsm6 libxext6 -y
# %pip install opencv-python
import cv2
from glob import glob
import os

## Extract 28 frames from each video (4 frames per second)

In [37]:

%cd /workspace/C684_Assignment/Complete_Sources
%mkdir -p ../processing/frames

video_filenames = []
video_fps = None
videos = glob(os.path.join("*.webm"))
video_dir = os.path.join("../processing/frames")
global_counter = 0
batch_dir = None
batch_size = 196
current_batch_no = -1
for v in videos:
    new_batch_no = global_counter // batch_size
    if current_batch_no < new_batch_no:
        current_batch_no = new_batch_no
        print("Starting batch %d" % current_batch_no)
        batch_dir = os.path.join(video_dir, "batch_%d" % current_batch_no)
        os.mkdir(batch_dir)
    v_noext = os.path.splitext(v)[0]
    video_filenames.append(v_noext)

    cap = cv2.VideoCapture(v) # capturing the video from the given path
    video_fps = cap.get(cv2.cv2.CAP_PROP_FPS)
    frames_per_quarter_sec = int(video_fps) / 4
    if (not cap.isOpened()):
        raise Exception("Cannot open file %s" % v)
    written_frame_count = 0
    current_frame = -1
    while(cap.isOpened()):
        current_frame += 1
        ret, frame = cap.read()
        if (ret != True):
            break
            
        if current_frame % frames_per_quarter_sec != 0:
            continue

        filename = os.path.join(batch_dir, "%s_%d.jpg" % (v_noext, current_frame))
        cv2.imwrite(filename, frame)
        written_frame_count += 1
    global_counter += written_frame_count
    cap.release()

/C684_Assignment/Complete_Sources
Starting batch 0
Starting batch 1
Starting batch 2
Starting batch 3
Starting batch 4
Starting batch 5
Starting batch 6
Starting batch 7
Starting batch 8
Starting batch 9
Starting batch 10
Starting batch 11
Starting batch 12
Starting batch 13
Starting batch 14
Starting batch 15
Starting batch 16
Starting batch 17
Starting batch 18
Starting batch 19
Starting batch 20
Starting batch 21
Starting batch 22
Starting batch 23
Starting batch 24
Starting batch 25
Starting batch 26
Starting batch 27
Starting batch 28
Starting batch 29
Starting batch 30
Starting batch 31
Starting batch 32
Starting batch 33
Starting batch 34
Starting batch 35
Starting batch 36
Starting batch 37
Starting batch 38
Starting batch 39
Starting batch 40
Starting batch 41
Starting batch 42
Starting batch 43
Starting batch 44
Starting batch 45
Starting batch 46
Starting batch 47
Starting batch 48
Starting batch 49
Starting batch 50
Starting batch 51
Starting batch 52
Starting batch 53
Star

Starting batch 436
Starting batch 437
Starting batch 438
Starting batch 439
Starting batch 440
Starting batch 441
Starting batch 442
Starting batch 443
Starting batch 444
Starting batch 445
Starting batch 446
Starting batch 447
Starting batch 448
Starting batch 449
Starting batch 450
Starting batch 451
Starting batch 452
Starting batch 453
Starting batch 454
Starting batch 455
Starting batch 456
Starting batch 457
Starting batch 458
Starting batch 459
Starting batch 460
Starting batch 461
Starting batch 462
Starting batch 463
Starting batch 464
Starting batch 465
Starting batch 466
Starting batch 467
Starting batch 468
Starting batch 469
Starting batch 470
Starting batch 471
Starting batch 472
Starting batch 473
Starting batch 474
Starting batch 475
Starting batch 476
Starting batch 477
Starting batch 478
Starting batch 479
Starting batch 480
Starting batch 481
Starting batch 482
Starting batch 483
Starting batch 484
Starting batch 485
Starting batch 486
Starting batch 487
Starting bat

Starting batch 868
Starting batch 869
Starting batch 870
Starting batch 871
Starting batch 872
Starting batch 873
Starting batch 874
Starting batch 875
Starting batch 876
Starting batch 877
Starting batch 878
Starting batch 879
Starting batch 880
Starting batch 881
Starting batch 882
Starting batch 883
Starting batch 884
Starting batch 885
Starting batch 886
Starting batch 887
Starting batch 888
Starting batch 889
Starting batch 890
Starting batch 891
Starting batch 892
Starting batch 893
Starting batch 894
Starting batch 895
Starting batch 896
Starting batch 897
Starting batch 898
Starting batch 899
Starting batch 900
Starting batch 901
Starting batch 902
Starting batch 903
Starting batch 904
Starting batch 905
Starting batch 906
Starting batch 907
Starting batch 908
Starting batch 909
Starting batch 910
Starting batch 911
Starting batch 912
Starting batch 913
Starting batch 914
Starting batch 915
Starting batch 916
Starting batch 917
Starting batch 918
Starting batch 919
Starting bat

## In PyTorch container, generate AMNet attention maps using the following script

```
#!/bin/bash
cd AMNet
export b="${1:-0}"
while [ -d "/C684_Assignment/processing/frames/batch_$b" ]
do
  echo "Trying Batch No: $b"
  timeout 120s bash -c 'python3 main.py --cnn ResNet50FC --model-weights data/lamem_ResNet50FC_lstm3_train_5/weights_35.pkl --eval-images /C684_Assignment/processing/frames/batch_$b --att-maps-out /C684_Assignment/processing/att_maps --test-batch-size 200 --csv-out /C684_Assignment/processing/amnet_pred_$b.csv' && b=$(($b + 1))
done
```

## Optional: Create a demo video from the heat maps

In [23]:
%cd /workspace/C684_Assignment/processing/att_maps

demo_video = "video10001"
print("Reading frames: %s" % demo_video)
frames = {}
frame_files = glob(demo_video + "*.jpg")
size = None
for f in frame_files:
    index = int(f.split("_")[1])
    img = cv2.imread(f)
    height, width, layers = img.shape
    size = (width,height)
    frames[index] = img
os.chdir("..")

output_file = "../att_maps-%s.avi" % video
print("Writing to file: %s" % output_file)
fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter(output_file, fourcc, 6, size)
for i in sorted(frames):
    out.write(frames[i])
out.release()

/workspace/C684_Assignment/processing/att_maps
Reading frames: video10001
Writing to file: ../att_maps-video10001.avi
